- 용인시 행정동 2개에 대한 set-covering optimization을 통해 최적의 입지 갯수 p개 설정 
- 기준은 Set-covering모형은 수요지와 공공시설 간에 최대 허용 거리(450m)를 미리 정해놓고, 그 조건을 만족시키는 범위 내에서 설치할 전기차 충전소 입지의 개수를 최소로 하는 것이 목적 (고객의 접근성 요소 고려)


In [57]:
import pandas as pd
import numpy as np
import random
import gurobipy as gp
from gurobipy import GRB


### Input Data

In [58]:
path = 'C:/bigcon'
jukjun1_od = pd.read_csv(path + 'output\data\checkpoint\jukjun1_od_mat.csv', encoding = 'utf-8')

In [59]:
# demand 따로 추출 
jukjun1_demand = jukjun1_od[['앱실행수']]
jukjun1_demand

,앱실행수
0,11015.605
1,10843.670
2,7606.771
3,6591.808
4,5416.644
5,5143.311
6,5127.690
7,4517.616
8,4352.650
9,4199.244


In [60]:
jukjun1_od_drop = jukjun1_od.drop(['앱실행수'],axis = 1)
jukjun1_od_drop

,Unnamed: 0,꽃메마을 한라신영프로방스,꽃메마을 현대홈타운 (4차 4단지),꽃메마을 현대홈타운 (4차 3단지),꽃메마을 극동스타클래스,꽃메마을 금강아파트,꽃메마을 아이파크,꽃메마을 한라프로방스 Ⅱ,도담마을 푸르지오아파트(601-602동),꽃메마을 현대홈타운 (4차2단지),...,새터마을 푸르지오아파트,대지마을 중앙하이츠빌,죽전 현대1차아파트,대지마을 현대홈타운 3차 2단지,내대지마을 정석베르데파크아파트,내대지마을 광명샤인빌,내대지마을 진흥더블파크,새터마을 죽전힐스테이트,내대지마을 건영캐스빌,죽전1동
0,0,1599.296758,1426.537077,1304.948730,1232.246850,1369.660586,1158.431465,1162.672420,1301.883713,1158.115519,...,296.735328,496.692844,780.302145,660.667641,601.888742,925.502337,843.203121,183.828968,365.641395,788.311686
1,1,369.724799,216.192061,239.996248,536.015489,140.907708,386.591958,290.921814,112.858602,79.915140,...,1018.752461,899.778474,1603.982867,1003.596894,1514.319465,1776.032860,1720.416193,1126.933451,1506.391192,797.822254
2,2,413.864442,405.726408,503.507075,803.540744,266.807665,650.172693,554.867425,183.651042,255.683221,...,1119.866513,866.835802,1764.477628,909.730348,1654.107927,1939.095474,1876.596949,1145.891706,1596.250441,683.725501
3,3,2473.704644,2250.338839,2062.479264,1781.616029,2274.008708,1838.873718,1910.333662,2252.442775,2104.036853,...,1295.918198,1801.725268,591.860260,1999.812646,755.075528,427.686838,508.730408,1528.663501,1018.972518,2078.501148
4,4,1960.326663,1737.468548,1550.736288,1279.020345,1759.785573,1328.008044,1397.509631,1738.678339,1590.818874,...,835.029507,1356.794125,222.791413,1570.030489,391.037980,248.063915,271.731225,1122.670454,714.117965,1617.542631
5,5,1102.848384,913.426727,754.791651,465.726268,1027.260306,626.064625,712.846566,1079.160102,1014.688043,...,1246.754559,1523.088010,1389.883225,1721.159747,1417.440591,1511.513926,1499.991190,1558.711014,1610.161620,1592.525355
6,6,677.779493,668.035089,734.222135,993.478061,531.070615,833.199132,744.174520,440.707903,446.217714,...,1068.852279,704.153736,1755.944144,692.770742,1624.236423,1930.226709,1860.679614,1013.064690,1514.839462,464.769057
7,7,963.377805,881.144305,870.291266,1029.041288,764.981345,877.044001,811.559658,672.550788,592.296414,...,780.849668,346.872883,1484.722021,349.852687,1336.036764,1655.230905,1580.368882,663.273331,1187.427928,129.607739
8,8,564.031074,339.108347,152.089622,234.544164,391.239714,84.652324,21.399329,411.715778,321.774652,...,918.212392,965.619932,1404.637185,1122.927220,1341.972182,1570.421742,1523.808635,1116.839506,1394.131223,953.133975
9,9,407.944460,185.550810,63.126508,372.227884,236.142991,241.581639,156.417057,272.236565,230.600083,...,1024.805335,996.139990,1549.473111,1127.264412,1478.214710,1717.369341,1668.117394,1185.773394,1508.764654,935.956447


In [61]:
# 출발지 기호 변경
depart_oldname = jukjun1_od_drop['Unnamed: 0'].unique().tolist()
depart_newname = [] # 바뀌는 값
for i in range(1,len(depart_oldname)+1):
    depart_newname.append('D' + str(i))

jukjun1_od_drop['수요지'] = jukjun1_od_drop['Unnamed: 0'].replace(depart_oldname,depart_newname) # 수요지 열 재생성
jukjun1_od_drop = jukjun1_od_drop.set_index(['수요지'])
jukjun1_od_drop.drop(['Unnamed: 0'],axis=1, inplace=True)

# 도착지 기호 변경
arrive_oldname = jukjun1_od_drop.columns.unique().tolist()
arrive_newname = [] # 바뀌는 값
for i in range(1,len(arrive_oldname)+1):
    arrive_newname.append('A' + str(i))

jukjun1_od_drop.columns = arrive_newname
jukjun1_od_drop

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A39,A40,A41,A42,A43,A44,A45,A46,A47,A48
수요지,,,,,,,,,,,,,,,,,,,,,
D1,1599.296758,1426.537077,1304.948730,1232.246850,1369.660586,1158.431465,1162.672420,1301.883713,1158.115519,1198.526129,...,296.735328,496.692844,780.302145,660.667641,601.888742,925.502337,843.203121,183.828968,365.641395,788.311686
D2,369.724799,216.192061,239.996248,536.015489,140.907708,386.591958,290.921814,112.858602,79.915140,111.210086,...,1018.752461,899.778474,1603.982867,1003.596894,1514.319465,1776.032860,1720.416193,1126.933451,1506.391192,797.822254
D3,413.864442,405.726408,503.507075,803.540744,266.807665,650.172693,554.867425,183.651042,255.683221,173.912546,...,1119.866513,866.835802,1764.477628,909.730348,1654.107927,1939.095474,1876.596949,1145.891706,1596.250441,683.725501
D4,2473.704644,2250.338839,2062.479264,1781.616029,2274.008708,1838.873718,1910.333662,2252.442775,2104.036853,2177.973205,...,1295.918198,1801.725268,591.860260,1999.812646,755.075528,427.686838,508.730408,1528.663501,1018.972518,2078.501148
D5,1960.326663,1737.468548,1550.736288,1279.020345,1759.785573,1328.008044,1397.509631,1738.678339,1590.818874,1665.448888,...,835.029507,1356.794125,222.791413,1570.030489,391.037980,248.063915,271.731225,1122.670454,714.117965,1617.542631
D6,1102.848384,913.426727,754.791651,465.726268,1027.260306,626.064625,712.846566,1079.160102,1014.688043,1087.871815,...,1246.754559,1523.088010,1389.883225,1721.159747,1417.440591,1511.513926,1499.991190,1558.711014,1610.161620,1592.525355
D7,677.779493,668.035089,734.222135,993.478061,531.070615,833.199132,744.174520,440.707903,446.217714,383.924388,...,1068.852279,704.153736,1755.944144,692.770742,1624.236423,1930.226709,1860.679614,1013.064690,1514.839462,464.769057
D8,963.377805,881.144305,870.291266,1029.041288,764.981345,877.044001,811.559658,672.550788,592.296414,575.906090,...,780.849668,346.872883,1484.722021,349.852687,1336.036764,1655.230905,1580.368882,663.273331,1187.427928,129.607739
D9,564.031074,339.108347,152.089622,234.544164,391.239714,84.652324,21.399329,411.715778,321.774652,398.442969,...,918.212392,965.619932,1404.637185,1122.927220,1341.972182,1570.421742,1523.808635,1116.839506,1394.131223,953.133975


In [63]:
# D6, D14 수요지는 450m 이내에 위치한 전기차 충전소 후보지가 없어 제외
jukjun1_od_drop.drop(['D6','D14'],axis=0,inplace=True)
jukjun1_od_drop

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,...,A39,A40,A41,A42,A43,A44,A45,A46,A47,A48
수요지,,,,,,,,,,,,,,,,,,,,,
D1,1599.296758,1426.537077,1304.948730,1232.246850,1369.660586,1158.431465,1162.672420,1301.883713,1158.115519,1198.526129,...,296.735328,496.692844,780.302145,660.667641,601.888742,925.502337,843.203121,183.828968,365.641395,788.311686
D2,369.724799,216.192061,239.996248,536.015489,140.907708,386.591958,290.921814,112.858602,79.915140,111.210086,...,1018.752461,899.778474,1603.982867,1003.596894,1514.319465,1776.032860,1720.416193,1126.933451,1506.391192,797.822254
D3,413.864442,405.726408,503.507075,803.540744,266.807665,650.172693,554.867425,183.651042,255.683221,173.912546,...,1119.866513,866.835802,1764.477628,909.730348,1654.107927,1939.095474,1876.596949,1145.891706,1596.250441,683.725501
D4,2473.704644,2250.338839,2062.479264,1781.616029,2274.008708,1838.873718,1910.333662,2252.442775,2104.036853,2177.973205,...,1295.918198,1801.725268,591.860260,1999.812646,755.075528,427.686838,508.730408,1528.663501,1018.972518,2078.501148
D5,1960.326663,1737.468548,1550.736288,1279.020345,1759.785573,1328.008044,1397.509631,1738.678339,1590.818874,1665.448888,...,835.029507,1356.794125,222.791413,1570.030489,391.037980,248.063915,271.731225,1122.670454,714.117965,1617.542631
D7,677.779493,668.035089,734.222135,993.478061,531.070615,833.199132,744.174520,440.707903,446.217714,383.924388,...,1068.852279,704.153736,1755.944144,692.770742,1624.236423,1930.226709,1860.679614,1013.064690,1514.839462,464.769057
D8,963.377805,881.144305,870.291266,1029.041288,764.981345,877.044001,811.559658,672.550788,592.296414,575.906090,...,780.849668,346.872883,1484.722021,349.852687,1336.036764,1655.230905,1580.368882,663.273331,1187.427928,129.607739
D9,564.031074,339.108347,152.089622,234.544164,391.239714,84.652324,21.399329,411.715778,321.774652,398.442969,...,918.212392,965.619932,1404.637185,1122.927220,1341.972182,1570.421742,1523.808635,1116.839506,1394.131223,953.133975
D10,407.944460,185.550810,63.126508,372.227884,236.142991,241.581639,156.417057,272.236565,230.600083,289.331511,...,1024.805335,996.139990,1549.473111,1127.264412,1478.214710,1717.369341,1668.117394,1185.773394,1508.764654,935.956447


In [64]:
# 죽전1동 arc 및 demand(P/T) dict
dic = {}

for i in jukjun1_od_drop.index:
    for name in jukjun1_od_drop.columns:
        key = (i, name)
        value = jukjun1_od_drop.loc[i, name]
        dic[key] = value

In [65]:
# 수요지와 도착지 
demand = {i for (i,j) in dic.keys()}
print("수요지 =",demand)

arrival = {j for (i,j) in dic.keys()}
print("전기차 충전소 후보지 =",arrival)

수요지 = {'D12', 'D3', 'D16', 'D20', 'D19', 'D1', 'D7', 'D10', 'D4', 'D5', 'D15', 'D17', 'D18', 'D8', 'D13', 'D2', 'D11', 'D9'}
전기차 충전소 후보지 = {'A48', 'A42', 'A9', 'A33', 'A15', 'A24', 'A17', 'A32', 'A16', 'A8', 'A23', 'A2', 'A28', 'A30', 'A44', 'A4', 'A34', 'A26', 'A13', 'A40', 'A12', 'A46', 'A6', 'A21', 'A19', 'A31', 'A39', 'A41', 'A14', 'A25', 'A37', 'A38', 'A3', 'A36', 'A45', 'A29', 'A1', 'A5', 'A20', 'A35', 'A27', 'A43', 'A10', 'A22', 'A7', 'A11', 'A18', 'A47'}


In [66]:
# 450m 이내에 있는 도착지 set 
close_arrival = { j : list() for j in demand}

for i in demand:
    for j in arrival:
        if dic[i,j] <= 450:
            close_arrival[i].append(j)
    print("수요지",i,"기준 450m 이내에 있는 후보지는",close_arrival[i])

수요지 D12 기준 450m 이내에 있는 후보지는 ['A43', 'A47']
수요지 D3 기준 450m 이내에 있는 후보지는 ['A9', 'A16', 'A8', 'A23', 'A2', 'A13', 'A21', 'A14', 'A1', 'A5', 'A20', 'A10', 'A18']
수요지 D16 기준 450m 이내에 있는 후보지는 ['A48', 'A9', 'A15', 'A24', 'A16', 'A23', 'A28', 'A30', 'A13', 'A12', 'A21', 'A31', 'A14', 'A38', 'A20', 'A27', 'A10', 'A18']
수요지 D20 기준 450m 이내에 있는 후보지는 ['A46', 'A39', 'A47']
수요지 D19 기준 450m 이내에 있는 후보지는 ['A48', 'A42', 'A28', 'A30', 'A40', 'A38']
수요지 D1 기준 450m 이내에 있는 후보지는 ['A32', 'A46', 'A31', 'A39', 'A36', 'A35', 'A47']
수요지 D7 기준 450m 이내에 있는 후보지는 ['A9', 'A24', 'A16', 'A8', 'A23', 'A28', 'A21', 'A14', 'A20', 'A10', 'A18']
수요지 D10 기준 450m 이내에 있는 후보지는 ['A9', 'A15', 'A8', 'A2', 'A4', 'A13', 'A12', 'A6', 'A3', 'A1', 'A5', 'A10', 'A7']
수요지 D4 기준 450m 이내에 있는 후보지는 ['A44']
수요지 D5 기준 450m 이내에 있는 후보지는 ['A44', 'A41', 'A37', 'A45', 'A29', 'A43', 'A22']
수요지 D15 기준 450m 이내에 있는 후보지는 ['A24', 'A32', 'A30', 'A40', 'A46', 'A19', 'A31', 'A39', 'A36', 'A35', 'A27']
수요지 D17 기준 450m 이내에 있는 후보지는 ['A33', 'A32', 'A34', 'A39', 'A

### Model deployment

In [67]:
model = gp.Model("electric_set_cover")

# variables
x = model.addVars(arrival, vtype = GRB.BINARY, name='x_j')

In [68]:
# objective function
obj = gp.quicksum(x[j] for j in arrival)
        
model.setObjective(obj, GRB.MINIMIZE)

In [69]:
# constraints

model.addConstrs(gp.quicksum(x[j] for j in close_arrival[i]) >= 1 for i in demand)


{'D12': <gurobi.Constr *Awaiting Model Update*>,
 'D3': <gurobi.Constr *Awaiting Model Update*>,
 'D16': <gurobi.Constr *Awaiting Model Update*>,
 'D20': <gurobi.Constr *Awaiting Model Update*>,
 'D19': <gurobi.Constr *Awaiting Model Update*>,
 'D1': <gurobi.Constr *Awaiting Model Update*>,
 'D7': <gurobi.Constr *Awaiting Model Update*>,
 'D10': <gurobi.Constr *Awaiting Model Update*>,
 'D4': <gurobi.Constr *Awaiting Model Update*>,
 'D5': <gurobi.Constr *Awaiting Model Update*>,
 'D15': <gurobi.Constr *Awaiting Model Update*>,
 'D17': <gurobi.Constr *Awaiting Model Update*>,
 'D18': <gurobi.Constr *Awaiting Model Update*>,
 'D8': <gurobi.Constr *Awaiting Model Update*>,
 'D13': <gurobi.Constr *Awaiting Model Update*>,
 'D2': <gurobi.Constr *Awaiting Model Update*>,
 'D11': <gurobi.Constr *Awaiting Model Update*>,
 'D9': <gurobi.Constr *Awaiting Model Update*>}

In [70]:
model.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 18 rows, 48 columns and 172 nonzeros
Model fingerprint: 0x1140171c
Variable types: 0 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 18 rows and 48 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 16 available processors)

Solution count 2: 4 6 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.000000000000e+00, best bound 4.000000000000e+00, gap 0.0000%


In [71]:
for v in model.getVars():
    if v.X != 0:
        print("%s : %f" % (v.Varname, v.X))

print(model.objVal) # 최적의 입지 갯수 P = 4개

x_j[A9] : 1.000000
x_j[A30] : 1.000000
x_j[A44] : 1.000000
x_j[A47] : 1.000000
4.0
